In [4]:
import numpy as np
import itertools
import pandas as pd
import os 

dims = np.concatenate([np.arange(2,10), np.ceil(np.logspace(2,5, 5)).astype(int)])

integrator_types = ['velocity_verlet', 'mclachlan', 'omelyan']

samplers = [[f"adjusted_microcanonical_{integrator_type}",
            
                f"unadjusted_microcanonical__{integrator_type}"] for integrator_type in integrator_types]

models = [f"Gaussian_{dim}" for dim in dims]

df = pd.concat([pd.read_csv(os.path.join('../..',f'results/{model}', f'{sampler}_{model}.csv')) for sampler, model in itertools.product(samplers, models)], ignore_index=True)

NameError: name 'integrator_type' is not defined